## Write a Software Controller Implementation

For now, you have to write the program directly in the target high-level AST. The language is somewhat close to Rust, although it is entirely expression-based (no statements). In this program, we

* write out controller. The control policy observes the pendulum angle and applies a proportional control policy to it.
* write our plant model. The plant is a 2D system, and we call the controller insider the function.

In [1]:
# write in the SEE-Reach target language
pendulum_program = """
fn pendulum_dynamics(theta: real, omega: real, kp: real, kd: real) -> tuple {
    // param theta: pendulum angle
    // param omega: angular rate
    // param kp: proportional gain
    // param kd: derivative gain
    
    // get the torque output
    let u: real = controller(theta, omega, kp, kd);
    
    // gravity and length parameters
    let g: real = -9.81;
    let l: real = 2.0;
    
    // state derivative
    let thetap: real = omega;
    let omegap: real = u + g / l * sin(theta);
    return (thetap, omegap)
}

fn controller(x: real, omega: real, kp: real, kd: real) -> real {
    // signal contributions from proportional and derivative
    let up: real = -1.0 * kp * x;
    let ud: real = -1.0 * kd * omega;
    let u: real = up + ud;
    
    // also, calculate the feedforward term
    let thetap: real = omega;
    let omegap: real = u + g / l * sin(theta);
    
    // we are torque limited--u must be in [-5.0, 5.0]
    if u < -5.0 {
        return -5.0
    } else {
        if u > 5.0 {
            return 5.0
        } else {
            return u
        }
    }
}
"""

In [2]:
from seereach.parser import SReachParser

# run the SEE-Reach parser
program = SReachParser.parse(pendulum_program)

## Symbolic Execution

In [3]:
# perform symbolic execution on the function
from seereach.fanalysis import function_symbolic_execution
from seereach.symlang import *
from seereach.lang import *

signature = [SVariable('theta', Type.REAL), SVariable('omega', Type.REAL)]

# can also pass in a mix of concrete and symbolic
signature = [
    SVariable('theta', Type.REAL), 
    SVariable('omega', Type.REAL),
    Literal(Value(Type.REAL, 1.0)), 
    Literal(Value(Type.REAL, 0.2))
]

r = function_symbolic_execution(program, 'pendulum_dynamics', signature )

KeyError: Name('g')

In [ ]:
# example analysis: simplify the contexts and print the system
from seereach.result import EvalResult
from seereach.sympyconvert import SymPyConverter
import sympy as sp

def simplify_evalresult(result: EvalResult):
    """apply a simplifier that connect the symbolic expression generated by the SEE to SymPy"""
    from functools import reduce
    spc = SymPyConverter()
    # AND all the path conditions together
    if len(result.path_condition) > 0:
        condition = [spc.simplify(reduce(lambda x, y: SBinaryOp(x, Operator.AND, y), result.path_condition))]
    else:
        condition  = [True]
    return EvalResult(
        spc.simplify(result.expr_eval),
        condition, 
        result.is_return,
    )

In [ ]:
from IPython.display import display, Math

# collect the lhs from the initial context
spc = SymPyConverter()
lhs = sp.latex(sp.Matrix([
    sp.symbols(v.name) for v in signature \
    if isinstance(v, SVariable)]))

# print the LaTeX equation
for result in r:
    rs = simplify_evalresult(result)
    if len(rs.path_condition) > 0:
        display(Math(spc.latex(rs.path_condition[0])))
    display(Math(lhs + "\\rightarrow" + spc.latex(rs.expr_eval)))